In [1]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

from sklearn.metrics import mean_absolute_error
import numpy as np


import os
import pandas as pd
from utils import (
    ESTACIONES_XLSX,
    NCEP_NCAR_XLSX,
    NDEFM_XLSX,
    PREDICTIONS_FOLDER
)

In [11]:
ncep_ncar_maximos = pd.read_excel(NCEP_NCAR_XLSX, sheet_name="Maximo", header=0)

In [53]:
df_maximos = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1, parse_dates=['Año hid'])
df_maximos = df_maximos[['Año hid', 'San Luis Tucuman']]
df_maximos = df_maximos[((~df_maximos['San Luis Tucuman'].isnull()).cumsum() == 0).idxmin():]

In [15]:
df_info_est = pd.read_excel(ESTACIONES_XLSX, sheet_name=0, header=0)
df_info_est = df_info_est[df_info_est.Estacion == 'San Luis Tucuman']
df_info_est[['lat', 'lon']] = -df_info_est['LAT (S), LONG (W)(º)'].str.split(',', expand=True).astype(float)
slt_loc = df_info_est[['lat', 'lon']].values[0]
slt_loc

array([-26.87, -65.02])

# Ayuda: ¿cómo elegir el punto de grilla para comparar con una determinada estación?

Si estamos trabajando, por ejemplo, con la estación San Luis cuya `lat = 26.06` y `lon = 66.0`, se debe seleccionar el punto de grilla más cercano a la estación. La latitud más cercana de la grilla es `25°`, la longitud más cercana de la grilla es `65°`, por lo tanto, se debe escoger el punto de grilla cuyo indicativo es `V13`.

In [58]:
ncep_ncar_closest = ncep_ncar_maximos[['año', 'V13']]
ncep_ncar_closest = ncep_ncar_closest[ncep_ncar_closest.año >= df_maximos.iloc[0]['Año hid'].year]
ncep_ncar_closest = ncep_ncar_closest[ncep_ncar_closest.año <= df_maximos.iloc[-1]['Año hid'].year]

In [59]:
mean_absolute_error(df_maximos[-10:]['San Luis Tucuman'], ncep_ncar_closest[-10:]['V13'])

50.80190493511898